In [1]:
# import csv
# import math
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import datetime as dt
from dateutil.relativedelta import relativedelta
# from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 

In [2]:
df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv")
df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])
df_allmatch.set_index("ID",inplace=True)
df_allmatch.sort_index(inplace=True)

In [3]:
df_allmatch

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRD,AwayRD
ID,,,,,,,,,,,,,,,,
6010524,2006,1,2006-03-05,yokohama-fa-marinos,kyoto-sanga,5,24,4,1,1,日産ス,23607,1500,1500,0,0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500,1500,0,0
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500,1500,0,0
6011001,2006,1,2006-03-05,sanfrecce-hiroshima,kashima-antlers,10,1,3,4,2,広島ビ,17564,1500,1500,0,0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500,1500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,三協Ｆ柏,5137,1519,1682,6,-13
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,味スタ,14373,1540,1453,8,0
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1453,1458,17,7


In [4]:
df_recent_stats = pd.read_csv("./recent_stats.csv", index_col=0)
df_recent_stats.fillna({
    'HomeShots':10, 'HomeShotsOnTarget':3, 'HomeShotsFromPK':0.2, 'HomePasses':400,
    'HomeCrosses':16, 'HomeDirectFK':10, 'HomeIndirectFK':2, 'HomeCK':4,
    'HomeThrowin':20, 'HomeDribbling':10, 'HomeTackles':18, 'HomeClearances':23,
    'HomeIntercepts':2, 'HomeOffsides':1.5, 'HomeYellowCards':1, 'HomeRedCards':0,
    'Home30mLineEntries':33, 'HomePenaltyAreaEntries':9, 'HomeAttacks':117,
    'HomeChanceBuildingRate':10, 'HomePossession':44,
    'AwayShots':10, 'AwayShotsOnTarget':3, 'AwayShotsFromPK':0.2, 'AwayPasses':400,
    'AwayCrosses':16, 'AwayDirectFK':10, 'AwayIndirectFK':2, 'AwayCK':4,
    'AwayThrowin':20, 'AwayDribbling':10, 'AwayTackles':18, 'AwayClearances':23,
    'AwayIntercepts':2, 'AwayOffsides':1.5, 'AwayYellowCards':1, 'AwayRedCards':0,
    'Away30mLineEntries':33, 'AwayPenaltyAreaEntries':9, 'AwayAttacks':117,
    'AwayChanceBuildingRate':10, 'AwayPossession':44}, inplace=True)
df_recent_stats.sort_index(inplace=True)

In [5]:
df_recent_stats

,HomeShots,HomeShotsOnTarget,HomeShotsFromPK,HomePasses,HomeCrosses,HomeDirectFK,HomeIndirectFK,HomeCK,HomeThrowin,HomeDribbling,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
10010103,10.0,3.0,0.2,400.0,16.0,10.0,2.0,4.0,20.0,10.0,...,23.0,2.0,1.5,1.0,0.0,33.0,9.0,117.0,10.00,44.00
10010908,10.0,3.0,0.2,400.0,16.0,10.0,2.0,4.0,20.0,10.0,...,23.0,2.0,1.5,1.0,0.0,33.0,9.0,117.0,10.00,44.00
10011007,10.0,3.0,0.2,400.0,16.0,10.0,2.0,4.0,20.0,10.0,...,23.0,2.0,1.5,1.0,0.0,33.0,9.0,117.0,10.00,44.00
10011229,10.0,3.0,0.2,400.0,16.0,10.0,2.0,4.0,20.0,10.0,...,23.0,2.0,1.5,1.0,0.0,33.0,9.0,117.0,10.00,44.00
10011354,10.0,3.0,0.2,400.0,16.0,10.0,2.0,4.0,20.0,10.0,...,23.0,2.0,1.5,1.0,0.0,33.0,9.0,117.0,10.00,44.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,11.2,3.4,0.0,471.8,15.4,10.4,2.4,3.0,24.6,9.2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,9.0,4.4,0.0,379.8,8.4,13.8,1.2,3.6,18.8,11.4,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,13.0,4.4,0.0,598.6,16.2,9.6,3.2,2.6,17.8,11.4,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,13.4,2.8,0.0,605.6,14.8,9.2,1.0,3.4,19.6,10.6,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


In [6]:
df = pd.concat([df_allmatch,df_recent_stats], axis=1)

In [7]:
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
6010524,2006,1,2006-03-05,yokohama-fa-marinos,kyoto-sanga,5,24,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011001,2006,1,2006-03-05,sanfrecce-hiroshima,kashima-antlers,10,1,3,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


In [8]:
items = ['Rate','RD','Shots', 'ShotsOnTarget', 'ShotsFromPK','Passes', 'Crosses', 'DirectFK', 'IndirectFK', 'CK','Throwin', 'Dribbling', 'Tackles', 'Clearances',
       'Intercepts', 'Offsides', 'YellowCards', 'RedCards','30mLineEntries', 'PenaltyAreaEntries', 'Attacks', 'Possession']
for item in items:
    df.insert(len(df.columns), f"{item}Diff", df[f"Home{item}"]-df[f"Away{item}"])

In [9]:
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,TacklesDiff,ClearancesDiff,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,PossessionDiff
6010524,2006,1,2006-03-05,yokohama-fa-marinos,kyoto-sanga,5,24,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011001,2006,1,2006-03-05,sanfrecce-hiroshima,kashima-antlers,10,1,3,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,2.4,9.0,0.4,-0.2,1.0,-0.2,-24.2,-12.2,8.6,-7.24
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,-1.2,7.4,-1.8,-0.8,0.0,0.0,-26.6,-1.4,8.6,-16.82
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,2.2,4.0,-0.4,1.2,-1.0,0.0,14.8,5.4,1.6,-2.00
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,-8.0,-5.2,-0.6,0.0,-1.6,-0.2,-10.0,-8.2,-17.6,1.76


In [10]:
match_id = 20345412

In [11]:
match = df[df.index==match_id]
match

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,TacklesDiff,ClearancesDiff,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,PossessionDiff
20345412,2020,34,2020-12-19,vegalta-sendai,shonan-bellmare,54,12,0,0,0,...,-6.2,2.0,0.6,0.4,0.4,0.0,-1.0,-1.8,-9.8,-6.46


In [12]:
match_year = match.at[match_id,"Year"]
match_year

2020

### ================

In [13]:
match.at[match_id,"Date"] - relativedelta(years=3)

Timestamp('2017-12-19 00:00:00')

In [14]:
df[((match.at[match_id,"Date"] - relativedelta(years=3)) <= df["Date"]) & (df["Date"] < match.at[match_id,"Date"] )]

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,TacklesDiff,ClearancesDiff,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,PossessionDiff
18010701,2018,1,2018-02-25,shimizu-s-pulse,kashima-antlers,7,1,0,0,0,...,-1.4,11.0,0.0,-1.0,-0.4,0.0,-31.2,-7.6,4.8,-9.30
18010908,2018,1,2018-02-24,gamba-osaka,nagoya-grampus-eight,9,8,2,3,2,...,-1.0,3.6,0.0,1.1,1.2,0.0,6.6,0.2,-3.4,6.52
18011014,2018,1,2018-02-24,sanfrecce-hiroshima,consadole-sapporo,10,14,1,0,1,...,6.0,-11.6,2.0,0.8,-1.8,0.0,13.4,3.6,5.2,7.64
18011247,2018,1,2018-02-24,shonan-bellmare,v-varen-nagasaki,12,47,2,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
18011321,2018,1,2018-02-25,jubilo-iwata,kawasaki-frontale,13,21,0,3,2,...,0.6,-6.8,0.2,-2.2,1.2,0.0,-7.0,-2.0,0.8,-3.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20331801,2020,33,2020-10-21,vissel-kobe,kashima-antlers,18,1,1,3,2,...,-3.6,11.2,0.4,0.6,-0.2,0.0,-6.2,-7.4,-10.4,1.36
20332033,2020,33,2020-12-16,cerezo-osaka,sagan-tosu,20,33,1,2,2,...,-1.2,2.0,-1.2,-1.8,0.4,0.0,-13.8,-2.4,-10.6,-5.14
20332103,2020,33,2020-12-16,kawasaki-frontale,urawa-red-diamonds,21,3,3,1,1,...,-1.8,-3.0,0.4,0.0,-0.8,0.2,20.6,9.8,-4.0,2.52
20332214,2020,33,2020-11-11,fc-tokyo,consadole-sapporo,22,14,1,0,1,...,-0.2,-1.4,-0.6,-1.6,0.8,0.2,-7.4,-0.6,-0.8,-9.20


### ==========================

In [15]:
df_past = df[(match_year-3 <= df["Year"]) & (df["Year"] <= match_year-1)]
df_past

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,TacklesDiff,ClearancesDiff,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,PossessionDiff
17010122,2017,1,2017-02-25,kashima-antlers,fc-tokyo,1,22,0,1,2,...,10.4,-5.6,1.4,-0.4,-0.2,0.0,-3.6,3.6,11.4,1.66
17010503,2017,1,2017-02-25,yokohama-fa-marinos,urawa-red-diamonds,5,3,3,2,1,...,-2.0,6.4,0.8,0.4,-0.8,0.0,-23.0,-5.8,-0.2,-6.70
17010718,2017,1,2017-02-25,shimizu-s-pulse,vissel-kobe,7,18,0,1,2,...,-3.2,-2.4,-2.8,-0.9,-0.4,-0.2,-14.2,-6.6,-1.4,-7.58
17010928,2017,1,2017-02-26,gamba-osaka,ventforet-kofu,9,28,1,1,0,...,10.4,3.0,2.6,0.0,1.2,0.2,4.4,-2.8,5.8,7.40
17011078,2017,1,2017-02-25,sanfrecce-hiroshima,albirex-niigata,10,78,1,1,0,...,-6.0,-9.0,-0.8,1.6,-0.4,-0.4,9.2,6.4,-15.8,11.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19341054,2019,34,2019-12-07,sanfrecce-hiroshima,vegalta-sendai,10,54,1,0,1,...,-0.4,-13.6,0.8,0.6,0.6,0.0,24.0,11.6,8.0,20.98
19341421,2019,34,2019-12-07,consadole-sapporo,kawasaki-frontale,14,21,1,2,2,...,-7.0,-4.8,0.6,0.2,-0.2,0.0,-6.0,2.0,5.8,-0.58
19341813,2019,34,2019-12-07,vissel-kobe,jubilo-iwata,18,13,4,1,1,...,3.8,-8.8,-0.8,0.6,-2.0,0.0,15.0,-0.6,1.6,1.00
19343120,2019,34,2019-12-07,oita-trinita,cerezo-osaka,31,20,0,2,2,...,2.8,-9.6,0.4,-0.6,0.8,0.0,8.2,-4.8,-13.6,1.56


In [16]:
X_train=df_past[[
    'Sec', 'Attendances', 'HomeRate', 'AwayRate',
    'HomeRD', 'AwayRD', 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
    'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
    'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
    'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
    'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
    'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
    'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
    'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
    'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
    'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
    'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayChanceBuildingRate',
    'AwayPossession','RateDiff', 'RDDiff', 'ShotsDiff',
    'ShotsOnTargetDiff', 'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff',
    'DirectFKDiff', 'IndirectFKDiff', 'CKDiff', 'ThrowinDiff',
    'DribblingDiff', 'TacklesDiff', 'ClearancesDiff', 'InterceptsDiff',
    'OffsidesDiff', 'YellowCardsDiff', 'RedCardsDiff', '30mLineEntriesDiff',
    'PenaltyAreaEntriesDiff', 'AttacksDiff', 'PossessionDiff'
]].values
y_train = df_past["W/L"].values

In [17]:
X_test = match[[
    'Sec', 'Attendances', 'HomeRate', 'AwayRate',
    'HomeRD', 'AwayRD', 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
    'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
    'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
    'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
    'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
    'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
    'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
    'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
    'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
    'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
    'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayChanceBuildingRate',
    'AwayPossession','RateDiff', 'RDDiff', 'ShotsDiff',
    'ShotsOnTargetDiff', 'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff',
    'DirectFKDiff', 'IndirectFKDiff', 'CKDiff', 'ThrowinDiff',
    'DribblingDiff', 'TacklesDiff', 'ClearancesDiff', 'InterceptsDiff',
    'OffsidesDiff', 'YellowCardsDiff', 'RedCardsDiff', '30mLineEntriesDiff',
    'PenaltyAreaEntriesDiff', 'AttacksDiff', 'PossessionDiff'
]].values
y_test = match["W/L"].values

In [18]:
params = {'n_estimators': [10, 30, 50, 100, 200, 300],'max_depth': [2, 3, 4, 5, 6, 7, 8]}
clf = GridSearchCV(RandomForestClassifier(random_state=1234), params, cv=5, n_jobs=-1, verbose=1)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1234),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [10, 30, 50, 100, 200, 300]},
             verbose=1)

In [19]:
clf.best_params_

{'max_depth': 7, 'n_estimators': 200}

In [20]:
clf.predict(X_test)[0]

1

In [21]:
clf.predict_proba(X_test)[0]

array([0.23136071, 0.42729783, 0.34134147])

In [22]:
def predict(match_id):
    df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv")
    df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])
    df_allmatch.set_index("ID",inplace=True)
    df_allmatch.sort_index(inplace=True)
    
    df_recent_stats = pd.read_csv("./recent_stats.csv", index_col=0)
    df_recent_stats.fillna({
        'HomeShots':10, 'HomeShotsOnTarget':3, 'HomeShotsFromPK':0.2, 'HomePasses':400,
        'HomeCrosses':16, 'HomeDirectFK':10, 'HomeIndirectFK':2, 'HomeCK':4,
        'HomeThrowin':20, 'HomeDribbling':10, 'HomeTackles':18, 'HomeClearances':23,
        'HomeIntercepts':2, 'HomeOffsides':1.5, 'HomeYellowCards':1, 'HomeRedCards':0,
        'Home30mLineEntries':33, 'HomePenaltyAreaEntries':9, 'HomeAttacks':117,
        'HomeChanceBuildingRate':10, 'HomePossession':44,
        'AwayShots':10, 'AwayShotsOnTarget':3, 'AwayShotsFromPK':0.2, 'AwayPasses':400,
        'AwayCrosses':16, 'AwayDirectFK':10, 'AwayIndirectFK':2, 'AwayCK':4,
        'AwayThrowin':20, 'AwayDribbling':10, 'AwayTackles':18, 'AwayClearances':23,
        'AwayIntercepts':2, 'AwayOffsides':1.5, 'AwayYellowCards':1, 'AwayRedCards':0,
        'Away30mLineEntries':33, 'AwayPenaltyAreaEntries':9, 'AwayAttacks':117,
        'AwayChanceBuildingRate':10, 'AwayPossession':44
    }, inplace=True)
    df_recent_stats.sort_index(inplace=True)
    
    df = pd.concat([df_allmatch,df_recent_stats], axis=1)
    items = [
        'Rate','RD','Shots', 'ShotsOnTarget', 'ShotsFromPK','Passes', 'Crosses', 'DirectFK', 
        'IndirectFK', 'CK','Throwin', 'Dribbling', 'Tackles', 'Clearances','Intercepts', 'Offsides', 
        'YellowCards', 'RedCards','30mLineEntries', 'PenaltyAreaEntries', 'Attacks', 'Possession'
    ]
    
    for item in items:
        df.insert(len(df.columns), f"{item}Diff", df[f"Home{item}"]-df[f"Away{item}"])
    
    match = df[df.index==match_id]
    match_year = match.at[match_id,"Year"]
    df_past = df[(match_year-3 <= df["Year"]) & (df["Year"] <= match_year-1)]
    
    X_train=df_past[[
        'Sec', 'Attendances', 'HomeRate', 'AwayRate',
        'HomeRD', 'AwayRD', 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
        'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
        'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
        'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
        'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
        'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
        'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
        'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
        'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
        'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
        'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayChanceBuildingRate',
        'AwayPossession','RateDiff', 'RDDiff', 'ShotsDiff',
        'ShotsOnTargetDiff', 'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff',
        'DirectFKDiff', 'IndirectFKDiff', 'CKDiff', 'ThrowinDiff',
        'DribblingDiff', 'TacklesDiff', 'ClearancesDiff', 'InterceptsDiff',
        'OffsidesDiff', 'YellowCardsDiff', 'RedCardsDiff', '30mLineEntriesDiff',
        'PenaltyAreaEntriesDiff', 'AttacksDiff', 'PossessionDiff'
    ]].values
    y_train = df_past["W/L"].values
    
    X_test = match[[
        'Sec', 'Attendances', 'HomeRate', 'AwayRate',
        'HomeRD', 'AwayRD', 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
        'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
        'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
        'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
        'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
        'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
        'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
        'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
        'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
        'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
        'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayChanceBuildingRate',
        'AwayPossession','RateDiff', 'RDDiff', 'ShotsDiff',
        'ShotsOnTargetDiff', 'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff',
        'DirectFKDiff', 'IndirectFKDiff', 'CKDiff', 'ThrowinDiff',
        'DribblingDiff', 'TacklesDiff', 'ClearancesDiff', 'InterceptsDiff',
        'OffsidesDiff', 'YellowCardsDiff', 'RedCardsDiff', '30mLineEntriesDiff',
        'PenaltyAreaEntriesDiff', 'AttacksDiff', 'PossessionDiff'
    ]].values
    y_test = match["W/L"].values
    
    params = {'n_estimators': [10, 30, 50, 100, 200, 300],'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10]}
    clf = GridSearchCV(RandomForestClassifier(random_state=1234), params, cv=5, n_jobs=-1, verbose=1)
    clf.fit(X_train,y_train)
    
    print(clf.best_params_)
    return clf.predict_proba(X_test)[0]


In [23]:
predict(13340711)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'max_depth': 4, 'n_estimators': 100}


array([0.26781669, 0.37033023, 0.36185308])

In [24]:
def predict2(match_id):
    df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv")
    df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])
    df_allmatch.set_index("ID",inplace=True)
    df_allmatch.sort_index(inplace=True)
    
    df_recent_stats = pd.read_csv("./recent_stats.csv", index_col=0)
    df_recent_stats.fillna({
        'HomeShots':10, 'HomeShotsOnTarget':3, 'HomeShotsFromPK':0.2, 'HomePasses':400,
        'HomeCrosses':16, 'HomeDirectFK':10, 'HomeIndirectFK':2, 'HomeCK':4,
        'HomeThrowin':20, 'HomeDribbling':10, 'HomeTackles':18, 'HomeClearances':23,
        'HomeIntercepts':2, 'HomeOffsides':1.5, 'HomeYellowCards':1, 'HomeRedCards':0,
        'Home30mLineEntries':33, 'HomePenaltyAreaEntries':9, 'HomeAttacks':117,
        'HomeChanceBuildingRate':10, 'HomePossession':44,
        'AwayShots':10, 'AwayShotsOnTarget':3, 'AwayShotsFromPK':0.2, 'AwayPasses':400,
        'AwayCrosses':16, 'AwayDirectFK':10, 'AwayIndirectFK':2, 'AwayCK':4,
        'AwayThrowin':20, 'AwayDribbling':10, 'AwayTackles':18, 'AwayClearances':23,
        'AwayIntercepts':2, 'AwayOffsides':1.5, 'AwayYellowCards':1, 'AwayRedCards':0,
        'Away30mLineEntries':33, 'AwayPenaltyAreaEntries':9, 'AwayAttacks':117,
        'AwayChanceBuildingRate':10, 'AwayPossession':44
    }, inplace=True)
    df_recent_stats.sort_index(inplace=True)
    
    df = pd.concat([df_allmatch,df_recent_stats], axis=1)
    items = [
        'Rate','RD','Shots', 'ShotsOnTarget', 'ShotsFromPK','Passes', 'Crosses', 'DirectFK', 
        'IndirectFK', 'CK','Throwin', 'Dribbling', 'Tackles', 'Clearances','Intercepts', 'Offsides', 
        'YellowCards', 'RedCards','30mLineEntries', 'PenaltyAreaEntries', 'Attacks', 'Possession'
    ]
    
    for item in items:
        df.insert(len(df.columns), f"{item}Diff", df[f"Home{item}"]-df[f"Away{item}"])
    
    match = df[df.index==match_id]
    match_year = match.at[match_id,"Year"]
    df_past = df[((match.at[match_id,"Date"] - relativedelta(years=3)) <= df["Date"]) & (df["Date"] < match.at[match_id,"Date"] )]
    
    X_train=df_past[[
        'Sec', 'Attendances', 'HomeRate', 'AwayRate',
        'HomeRD', 'AwayRD', 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
        'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
        'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
        'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
        'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
        'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
        'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
        'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
        'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
        'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
        'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayChanceBuildingRate',
        'AwayPossession','RateDiff', 'RDDiff', 'ShotsDiff',
        'ShotsOnTargetDiff', 'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff',
        'DirectFKDiff', 'IndirectFKDiff', 'CKDiff', 'ThrowinDiff',
        'DribblingDiff', 'TacklesDiff', 'ClearancesDiff', 'InterceptsDiff',
        'OffsidesDiff', 'YellowCardsDiff', 'RedCardsDiff', '30mLineEntriesDiff',
        'PenaltyAreaEntriesDiff', 'AttacksDiff', 'PossessionDiff'
    ]].values
    y_train = df_past["W/L"].values
    
    X_test = match[[
        'Sec', 'Attendances', 'HomeRate', 'AwayRate',
        'HomeRD', 'AwayRD', 'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK',
        'HomePasses', 'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
        'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
        'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
        'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
        'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
        'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
        'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
        'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
        'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
        'AwayPenaltyAreaEntries', 'AwayAttacks', 'AwayChanceBuildingRate',
        'AwayPossession','RateDiff', 'RDDiff', 'ShotsDiff',
        'ShotsOnTargetDiff', 'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff',
        'DirectFKDiff', 'IndirectFKDiff', 'CKDiff', 'ThrowinDiff',
        'DribblingDiff', 'TacklesDiff', 'ClearancesDiff', 'InterceptsDiff',
        'OffsidesDiff', 'YellowCardsDiff', 'RedCardsDiff', '30mLineEntriesDiff',
        'PenaltyAreaEntriesDiff', 'AttacksDiff', 'PossessionDiff'
    ]].values
    y_test = match["W/L"].values
    
    params = {'n_estimators': [30, 50, 100, 200, 300],'max_depth': [2, 3, 4, 5, 6, 7, 8 , 9,10]}
    clf = GridSearchCV(RandomForestClassifier(random_state=1234), params, cv=5, n_jobs=-1, verbose=1)
    clf.fit(X_train,y_train)
    
    print(clf.best_params_)
    return clf.predict_proba(X_test)[0]


In [26]:
predict2(20300831)

Fitting 5 folds for each of 45 candidates, totalling 225 fits
{'max_depth': 7, 'n_estimators': 30}


array([0.21744667, 0.33812381, 0.44442952])

---

In [94]:
import numpy as np
import pandas as pd
from scipy.stats import poisson
from dateutil.relativedelta import relativedelta

In [95]:
df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])
df_allmatch

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRD,AwayRD
ID,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500,1500,0,0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500,1500,0,0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500,1500,0,0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500,1500,0,0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500,1500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1594,1560,4,-6
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1515,1524,9,-13
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1453,1458,17,7


In [96]:
def predict_with_poisson(match_id, match_range = 17):
    
    df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
    df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])

    current_day = df_allmatch.at[match_id,"Date"]
    home = df_allmatch.at[match_id,"Home"]
    away = df_allmatch.at[match_id,"Away"]
    
    df_recent_home = df_allmatch[
        (df_allmatch["Date"] < current_day) &
        ((current_day.year-1)<= df_allmatch["Year"]) &
        (df_allmatch["Home"] == home) 
    ].tail(match_range)

    if  (match_range- len(df_recent_home)) :
        for _ in range(match_range- len(df_recent_home)):
            df_recent_home = df_recent_home.append({'Date':(current_day-relativedelta(days=1)) , 'HomeGF': 0.99, 'AwayGF': 1.66}, ignore_index=True)

    df_recent_away = df_allmatch[
        (df_allmatch["Date"] < current_day) &
        ((current_day.year-1)<= df_allmatch["Year"]) &
        (df_allmatch["Away"] == away)
    ].tail(match_range)

    if  (match_range- len(df_recent_away)) :
        for _ in range(match_range- len(df_recent_away)):
            df_recent_away = df_recent_away.append({'Date':(current_day-relativedelta(days=1)) , 'HomeGF': 1.83, 'AwayGF': 0.94}, ignore_index=True)

    beginning_day = min(df_recent_home.iloc[0,2],df_recent_away.iloc[0,2])

    df_recent = df_allmatch[
        (beginning_day <= df_allmatch["Date"] ) &
        (df_allmatch["Date"] < current_day) 
    ]

    homegf_ave = df_recent_home["HomeGF"].mean()
    homega_ave = df_recent_home["AwayGF"].mean()
    awaygf_ave = df_recent_away["AwayGF"].mean()
    awayga_ave = df_recent_away["HomeGF"].mean()
    homegf_league_ave = df_recent["HomeGF"].mean()
    awaygf_league_ave = df_recent["AwayGF"].mean()
    
    home_attack = homegf_ave / homegf_league_ave
    away_defense = awayga_ave / homegf_league_ave
    homegf_pred = home_attack * away_defense * homegf_league_ave

    away_attack = awaygf_ave / awaygf_league_ave
    home_defense = homega_ave / awaygf_league_ave
    awaygf_pred = away_attack * home_defense * awaygf_league_ave
    
    x =  np.arange(0, 10, 1)

    home_poisson = poisson.pmf(x, homegf_pred)
    away_poisson = poisson.pmf(x, awaygf_pred)

    df_poisson = pd.DataFrame([home_poisson,away_poisson],columns=x, index=[home,away])
    
    win_prob_sum = 0
    loss_prob_sum = 0
    draw_prob_sum = 0
    
    for gf in x:
        gf_prob = df_poisson.at[home, gf]
        for ga in x:
            ga_prob = df_poisson.at[away,ga]
            score_prob = gf_prob * ga_prob

            if gf>ga:
                win_prob_sum += score_prob
            elif gf<ga:
                loss_prob_sum += score_prob
            else:
                draw_prob_sum += score_prob

    prob_sum = win_prob_sum+loss_prob_sum+draw_prob_sum
    win_prob_sum = win_prob_sum/prob_sum
    loss_prob_sum = loss_prob_sum/prob_sum
    draw_prob_sum = draw_prob_sum/prob_sum

    return draw_prob_sum,win_prob_sum,loss_prob_sum



In [97]:
predict_with_poisson(20300831)

(0.21909828284933852, 0.6407373050858897, 0.14016441206477182)

In [98]:
np.set_printoptions(suppress=True)

In [99]:
def predict_goalfor_with_poisson(match_id, match_range = 17):
    
    df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
    df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])
    
    current_day = df_allmatch.at[match_id,"Date"]
    home = df_allmatch.at[match_id,"Home"]
    away = df_allmatch.at[match_id,"Away"]
    
    df_recent_home = df_allmatch[
        (df_allmatch["Date"] < current_day) &
        ((current_day.year-1)<= df_allmatch["Year"]) &
        (df_allmatch["Home"] == home) 
    ].tail(match_range)

    if  (match_range- len(df_recent_home)) :
        for _ in range(match_range- len(df_recent_home)):
            df_recent_home = df_recent_home.append({'Date':(current_day-relativedelta(days=1)) , 'HomeGF': 0.99, 'AwayGF': 1.66}, ignore_index=True)

    df_recent_away = df_allmatch[
        (df_allmatch["Date"] < current_day) &
        ((current_day.year-1)<= df_allmatch["Year"]) &
        (df_allmatch["Away"] == away)
    ].tail(match_range)

    if  (match_range- len(df_recent_away)) :
        for _ in range(match_range- len(df_recent_away)):
            df_recent_away = df_recent_away.append({'Date':(current_day-relativedelta(days=1)) , 'HomeGF': 1.83, 'AwayGF': 0.94}, ignore_index=True)

    beginning_day = min(df_recent_home.iloc[0,2],df_recent_away.iloc[0,2])

    df_recent = df_allmatch[
        (beginning_day <= df_allmatch["Date"] ) &
        (df_allmatch["Date"] < current_day) 
    ]

    homegf_ave = df_recent_home["HomeGF"].mean()
    homega_ave = df_recent_home["AwayGF"].mean()
    awaygf_ave = df_recent_away["AwayGF"].mean()
    awayga_ave = df_recent_away["HomeGF"].mean()
    homegf_league_ave = df_recent["HomeGF"].mean()
    awaygf_league_ave = df_recent["AwayGF"].mean()
    
    home_attack = homegf_ave / homegf_league_ave
    away_defense = awayga_ave / homegf_league_ave
    homegf_pred = home_attack * away_defense * homegf_league_ave

    away_attack = awaygf_ave / awaygf_league_ave
    home_defense = homega_ave / awaygf_league_ave
    awaygf_pred = away_attack * home_defense * awaygf_league_ave
    
    x =  np.arange(0, 10, 1)
    
    home_poisson = poisson.pmf(x, homegf_pred)
    away_poisson = poisson.pmf(x, awaygf_pred)
    
    return home_poisson,away_poisson

In [100]:
predict_goalfor_with_poisson(20300831)

(array([0.15883, 0.29223, 0.26884, 0.16489, 0.07585, 0.02791, 0.00856,
        0.00225, 0.00052, 0.00011]),
 array([0.4865 , 0.35053, 0.12628, 0.03033, 0.00546, 0.00079, 0.00009,
        0.00001, 0.     , 0.     ]))